參考文章1: http://hhcmax.blogspot.com/2008/03/triggerafterbefore.html

參考文章2: https://dotblogs.com.tw/jamesfu/2014/06/20/triggersample

# 簡介

<img src='img/16.png'>

# Trigger 的基本操作

In [ ]:
CREATE TABLE dbo.MMX
(
	A varchar(5) NULL,
	B int NULL
)

In [ ]:
INSERT INTO dbo.MMX
VALUES('A', 0),
      ('B', 1)      

GO

### 建立 After 的 insert trigger

In [ ]:
CREATE TRIGGER dbo.tI_MMX ON dbo.MMX
FOR INSERT
AS
PRINT 'Exec tI_MMX'
IF EXISTS(
			SELECT t1.A 
			FROM dbo.MMX as t1 with (nolock)
			INNER JOIN INSERTED t2 with (nolock)
			ON t1.A = t2.A
		`	GROUP BY t1.A
			HAVING COUNT(1) > 1
          )
BEGIN
	Raiserror ('資料重複',16,1)
	Return
End
GO

### 建立 Before 的 insert trigger

In [ ]:
CREATE TRIGGER dbo.tII_MMX on dbo.MMX
INSTEAD OF INSERT
AS
PRINT 'Exec tII_MMX'
IF EXISTS(
			SELECT t1.A 
			FROM dbo.MMX as t1 with (nolock)
			INNER JOIN INSERTED t2 with (nolock)
			ON t1.A = t2.A
          )
BEGIN
	Raiserror ('資料重複',16,1)
	Return
End
ELSE　INSERT INTO dbo.MMX 
      SELECT * FROM INSERTED
GO

---

In [ ]:
INSERT INTO dbo.MMX
VALUES('A', 8)

In [ ]:
INSERT INTO dbo.MMX
VALUES('C', 8)

### 建立一個 After 的 update trigger

In [ ]:
建立一個 After 的Update Trigger，用來防止A欄位資料重覆。

In [ ]:
CREATE TRIGGER dbo.tU_MMX on dbo.MMX
FOR UPDATE
AS
PRINT 'Exec tU_MMX'
IF EXISTS(
			SELECT t1.A 
			FROM dbo.MMX t1 with(nolock)
			INNER JOIN INSERTED t2 with(nolock) on t1.A = t2.A
			GROUP BY t1.A
			HAVING COUNT(1) > 1)

BEGIN
	Raiserror ('資料重複',16,1)
	Rollback transaction
	Return
END
GO

### 建立一個 Before 的 update trigger

In [ ]:
CREATE TRIGGER dbo.tUI_MMX on dbo.MMX
INSTEAD of UPDATE
AS
PRINT 'Exec tUI_MMX'

IF (SELECT COUNT(1) FROM DELETED) <> 1
	Return

DECLARE @OldID varchar(5), @NewID varchar(5)
SELECT @OldID = A FROM DELETED
SELECT @NewID = A  FROM INSERTED

IF EXISTS(SELECT A FROM dbo.MMX with (nolock) WHERE A = @NewID) AND (@OldID <> @NewID)
BEGIN
	Raiserror ('資料重複',16,1)
	Return
END
	
UPDATE t1
   SET A = t2.A,
       B = t2.B
   FROM dbo.MMX t1 INNER JOIN INSERTED t2
   ON t1.A = @OldID AND t2.A = @NewID

GO

---

In [ ]:
UPDATE dbo.MMX
   SET A = 'A'
 WHERE A = 'C'

In [ ]:
因 A 欄位資料未重覆所以在 dbo.tUI_MMX 中會實際去異動 dbo.MMX 而觸發 dbo.tU_MMX 的執行。

In [ ]:
 UPDATE dbo.MMX
    SET A = 'D' 
  WHERE A = 'C'

### 範例01

In [ ]:
CREATE TRIGGER take_off_insert_datetime_conversion_trigger ON dbo.take_off
INSTEAD OF INSERT
AS
PRINT '執行 take_off_insert_datetime_conversion_trigger'

IF EXISTS(SELECT 1 FROM INSERTED WHERE start_datetime is null)
BEGIN
	DECLARE @start datetimeoffset
	SELECT @start= 	ToDateTimeOffset(concat(start_year,'-', start_month, '-',start_day,' ',start_hour, ':', start_minute), DATEPART(TZOFFSET, SYSDATETIMEOFFSET())) FROM INSERTED
	DECLARE @end datetimeoffset
	SELECT @end = ToDateTimeOffset(concat(end_year,'-', end_month, '-',end_day,' ',end_hour, ':', end_minute), DATEPART(TZOFFSET, SYSDATETIMEOFFSET())) FROM INSERTED
	
	INSERT INTO dbo.take_off 
	   SELECT [district_prosecutors_serial_id], 
	          [hr_criminal_serial_id], 
			  [caseprj_serial_id], 
			  [start_year], 
			  [start_month], 
			  [start_day], 
			  [start_hour], 
			  [start_minute], 
			  [end_year], 
			  [end_month], 
			  [end_day], 
			  [end_hour], 
			  [end_minute], 
			  [reason], 
			  [note], 
			  [noticesystem_serial_id], 
			  [t_year], 
			  [t_month], 
			  [t_day], 
			  [t_hour], 
			  [t_minute], 
			  [t_sec], 
			  [str_take_off_date], 
			  [str_take_off_time], 
			  [end_take_off_date], 
			  [end_take_off_time], 
			  [insdt], 
			  [insuid], 
			  [insunm], 
			  [upddt], 
			  [upduid], 
			  [updunm], 
			  [ipaddress], 
			  @start, 
			  @end
	   FROM INSERTED

END

IF EXISTS(SELECT 1 FROM INSERTED WHERE start_year is null)
BEGIN
	INSERT INTO dbo.take_off 
	   SELECT [district_prosecutors_serial_id], 
	          [hr_criminal_serial_id], 
			  [caseprj_serial_id], 
			  DATEPART(YEAR, start_datetime), 
			  DATEPART(MONTH, start_datetime), 
			  DATEPART(DAY, start_datetime), 
			  DATEPART(HOUR, start_datetime), 
			  DATEPART(MINUTE, start_datetime), 
			  DATEPART(YEAR, end_datetime), 
			  DATEPART(MONTH, end_datetime), 
			  DATEPART(DAY, end_datetime), 
			  DATEPART(HOUR, end_datetime), 
			  DATEPART(SECOND, end_datetime), 
			  [reason], 
			  [note], 
			  [noticesystem_serial_id], 
			  [t_year], 
			  [t_month], 
			  [t_day], 
			  [t_hour], 
			  [t_minute], 
			  [t_sec], 
			  [str_take_off_date], 
			  [str_take_off_time], 
			  [end_take_off_date], 
			  [end_take_off_time], 
			  [insdt], 
			  [insuid], 
			  [insunm], 
			  [upddt], 
			  [upduid], 
			  [updunm], 
			  [ipaddress], 
			  [start_datetime],
			  [end_datetime]
	   FROM INSERTED

END
GO

### 範例02

In [ ]:
CREATE TRIGGER take_off_update_datetime_conversion_trigger ON dbo.take_off
INSTEAD OF UPDATE
AS
PRINT '執行 take_off_update_datetime_conversion_trigger'

DECLARE @old_start_year int,
		@old_start_month int,
	    @old_start_day int ,
		@old_start_hour int ,
		@old_start_minute int ,
		@old_end_year int ,
		@old_end_month int,
		@old_end_day int ,
		@old_end_hour int ,
		@old_end_minute int,
		@old_start_datetime datetimeoffset,
		@old_end_datetime datetimeoffset,
		@new_start_year int,
		@new_start_month int,
	    @new_start_day int ,
		@new_start_hour int ,
		@new_start_minute int ,
		@new_end_year int ,
		@new_end_month int,
		@new_end_day int ,
		@new_end_hour int ,
		@new_end_minute int,
	    @new_start_datetime datetimeoffset,
		@new_end_datetime datetimeoffset

SELECT @old_start_year =  start_year FROM DELETED
SELECT @old_start_month =  start_month FROM DELETED
SELECT @old_start_day =  start_day FROM DELETED
SELECT @old_start_hour =  start_hour FROM DELETED
SELECT @old_start_minute =  start_minute FROM DELETED
SELECT @old_end_year =  end_year FROM DELETED
SELECT @old_end_month =  end_month FROM DELETED
SELECT @old_end_day =  end_day FROM DELETED
SELECT @old_end_hour =  end_hour FROM DELETED
SELECT @old_end_minute =  end_minute FROM DELETED
SELECT @old_start_datetime = start_datetime FROM DELETED
SELECT @old_end_datetime = end_datetime FROM DELETED
SELECT @new_start_year =  start_year FROM INSERTED
SELECT @new_start_month =  start_month FROM INSERTED
SELECT @new_start_day =  start_day FROM INSERTED
SELECT @new_start_hour =  start_hour FROM INSERTED
SELECT @new_start_minute =  start_minute FROM INSERTED
SELECT @new_end_year =  end_year FROM INSERTED
SELECT @new_end_month =  end_month FROM INSERTED
SELECT @new_end_day =  end_day FROM INSERTED
SELECT @new_end_hour =  end_hour FROM INSERTED
SELECT @new_end_minute =  end_minute FROM INSERTED
SELECT @new_start_datetime = start_datetime FROM INSERTED
SELECT @new_end_datetime = end_datetime FROM INSERTED



IF (@old_start_year <> @new_start_year) OR (@old_start_month <> @new_start_month) OR (@old_start_day <> @new_start_day) OR (@old_start_hour <> @new_start_hour) OR (@old_start_minute <> @new_start_minute) OR 
   (@old_end_year <> @new_end_year) OR (@old_end_month <> @new_end_month) OR (@old_end_day <> @new_end_day) OR (@old_end_hour <> @new_end_hour) OR (@old_end_minute <> @new_end_minute) 

BEGIN
	UPDATE t
	SET t.start_year = i.start_year,
	    t.start_month = i.start_month,
		t.start_day = i.start_day,
		t.start_hour = i.start_hour,
		t.start_minute = i.start_minute,
		t.end_year = i.end_year,
		t.end_month = i.end_month,
		t.end_day = i.end_day,
		t.end_hour = i.end_hour,
		t.end_minute = i.end_minute,
		t.start_datetime = ToDateTimeOffset(concat(i.start_year,'-', i.start_month, '-',i.start_day,' ',i.start_hour, ':', i.start_minute), DATEPART(TZOFFSET, SYSDATETIMEOFFSET())),
		t.end_datetime = ToDateTimeOffset(concat(i.end_year,'-', i.end_month, '-',i.end_day,' ',i.end_hour, ':', i.end_minute), DATEPART(TZOFFSET, SYSDATETIMEOFFSET()))
	FROM dbo.take_off t join INSERTED i
	ON t.serial_id = i.serial_id
END

IF (@old_start_datetime <> @new_start_datetime) OR (@old_end_datetime <> @new_end_datetime)
BEGIN
	UPDATE t
	SET t.start_datetime = i.start_datetime,
	    t.end_datetime = i.end_datetime,
		t.start_year = DATEPART(YEAR, i.start_datetime),
		t.start_month = DATEPART(MONTH, i.start_datetime),
		t.start_day = DATEPART(DAY, i.start_datetime),
		t.start_hour = DATEPART(HOUR, i.start_datetime),
		t.start_minute = DATEPART(MINUTE, i.start_datetime),
		t.end_year = DATEPART(YEAR, i.end_datetime),
		t.end_month = DATEPART(MONTH, i.end_datetime),
		t.end_day = DATEPART(DAY, i.end_datetime),
		t.end_hour = DATEPART(HOUR, i.end_datetime),
		t.end_minute = DATEPART(MINUTE, i.end_datetime)
	FROM dbo.take_off t join INSERTED i
	ON t.serial_id = i.serial_id

END

UPDATE t
SET 
    t.district_prosecutors_serial_id = i.district_prosecutors_serial_id,
    t.hr_criminal_serial_id = i.hr_criminal_serial_id,
	t.caseprj_serial_id = i.caseprj_serial_id,
	t.reason = i.reason,
	t.note = i.note,
	t.noticesystem_serial_id = i.noticesystem_serial_id,
	t.t_year = i.t_year,
	t.t_month = i.t_month,
	t.t_day = i.t_day,
	t.t_hour = i.t_hour,
	t.t_minute = i.t_minute,
	t.t_sec = i.t_sec,
	t.str_take_off_date = i.str_take_off_date,
	t.str_take_off_time = i.str_take_off_time,
	t.end_take_off_date = i.end_take_off_date,
	t.end_take_off_time = i.end_take_off_time,
	t.insdt = i.insdt,
	t.insuid = i.insuid,
	t.insunm = i.insunm,
	t.upddt = i.upddt,
	t.upduid = i.upduid
FROM dbo.take_off t join INSERTED i
ON t.serial_id = i.serial_id

GO

### 範例03

In [ ]:
CREATE TRIGGER LLTDispatchList_insert_trigger ON LLTDispatchList
FOR INSERT
AS
PRINT '執行 LLTDispatchList_insert_trigger'

BEGIN
	INSERT INTO survey_dispatch(dispatch_list_disp_no, arrival_date)
			SELECT disp_no, CAST(concat(disp_year + 1911,'-', disp_month,'-', disp_day) as date)
			FROM INSERTED

END
GO

### 範例04

In [ ]:
CREATE TRIGGER noticesystem_update_trigger on noticesystem
FOR UPDATE
AS
PRINT '執行 noticesystem_update_trigger'

IF (SELECT subject_type_serial_id from DELETED) = 1  AND  (SELECT vio_flag FROM INSERTED) <> '' AND (SELECT update_notification from DELETED) is NULL
BEGIN
	
		DECLARE @UPDATE_ID INT
		SELECT @UPDATE_ID = serial_id from INSERTED

		INSERT INTO survey_information(noticesystem_serial_id, arrival_date, hr_prosecution_serial_id)
		SELECT n.serial_id, CAST(n.notice_date as date), a.owner_serial_id
		FROM noticesystem as n 
		JOIN ac_account as a on n.insuid = a.[user]
		WHERE n.serial_id = @UPDATE_ID
END
GO

### 範例05

In [ ]:
CREATE TRIGGER application_open_case_insert_trigger ON application_open_case
FOR INSERT
AS 
PRINT '執行 application_open_case_insert_trigger'


IF (SELECT current_state FROM INSERTED) = 'complete' 
BEGIN
	INSERT INTO hr_criminal(district_prosecutors_serial_id,
	                        name, 
							tel, 
							mobile, 
							address, 
							pcode, 
							birthday_year, 
							birthday_month, 
							birthday_day,
							birthday_address,
							sex_type,
							police_block,
							police_tel,
							jobs,
							house_type_serial_id,
							police_officer,
							height_cm,
							weight_kg,
							marriage,
							ankle_cm,
							house_ping,
							residence_address,
							beep_switch,
							police_mobile,
							application_open_case_serial_id)

	   SELECT h.district_prosecutors_serial_id,
	          i.hr_criminal_name,
			  i.hr_criminal_tel,
			  i.hr_criminal_mobile,
			  i.hr_criminal_address,
			  i.hr_criminal_pcode,
			  year(i.hr_criminal_birthday),
			  month(i.hr_criminal_birthday),
			  day(i.hr_criminal_birthday),
			  i.hr_criminal_address,
			  i.hr_criminal_sex_type,
			  i.hr_criminal_police_block,
			  i.hr_criminal_police_tel,
			  i.hr_criminal_jobs,
			  i.hr_criminal_house_type_serial_id,
			  i.hr_criminal_police_officer,
			  i.hr_criminal_height_cm,
			  i.hr_criminal_weight_kg,
			  i.hr_criminal_marriage,
			  i.hr_criminal_ankle_cm,
			  i.hr_criminal_house_ping,
			  i.hr_criminal_residence_address,
			  i.hr_criminal_beep_switch,
			  i.hr_criminal_police_mobile,
			  i.serial_id
	   FROM INSERTED as i
	   JOIN hr_prosecution as h on i.hr_prosecution_serial_id = h.serial_id 


WAITFOR DELAY '00:00:03'


INSERT INTO caseprj(hr_criminal_serial_id, 
                    district_prosecutors_serial_id,
                    hr_prosecution_serial_id,
                    datetime_start_year,
                    datetime_start_month,
                    datetime_start_day,
                    datetime_end_year,
                    datetime_end_month,
                    datetime_end_day,
                    tag_monitor_sec,
                    prosecutor_year,
                    prosecutor_type,
                    prosecutor_monitor,
                    datetime_start,
                    datetime_end,
                    electronic_fence,
                    prosecutor_number)

    SELECT h.serial_id,
           h.district_prosecutors_serial_id,
           i.hr_prosecution_serial_id,
           year(i.caseprj_datetime_start),
           month(i.caseprj_datetime_start),
           day(i.caseprj_datetime_start),
           year(i.caseprj_datetime_end),
           month(i.caseprj_datetime_end),
           day(i.caseprj_datetime_end),
           300,
           i.prosecutor_year,
           i.prosecutor_type,
           i.caseprj_monitor_time,
           i.caseprj_datetime_start,
           i.caseprj_datetime_end,
           CONCAT(i.caseprj_no_exit, ';', i.caseprj_no_entry, ';', i.caseprj_entry_notify),
           i.prosecutor_number

    FROM INSERTED as i
    JOIN hr_criminal as h ON i.serial_id = h.application_open_case_serial_id
    
END
GO

### 範例06

In [ ]:
CREATE TRIGGER application_open_case_update_trigger ON application_open_case
FOR UPDATE
AS 
PRINT '執行 application_open_case_update_trigger'


IF (SELECT current_state FROM INSERTED) = 'complete'  AND (SELECT current_state FROM DELETED) <> 'complete'
BEGIN
	INSERT INTO hr_criminal(district_prosecutors_serial_id,
	                        name, 
							tel, 
							mobile, 
							address, 
							pcode, 
							birthday_year, 
							birthday_month, 
							birthday_day,
							birthday_address,
							sex_type,
							police_block,
							police_tel,
							jobs,
							house_type_serial_id,
							police_officer,
							height_cm,
							weight_kg,
							marriage,
							ankle_cm,
							house_ping,
							residence_address,
							beep_switch,
							police_mobile,
							application_open_case_serial_id)

	   SELECT h.district_prosecutors_serial_id,
	          i.hr_criminal_name,
			  i.hr_criminal_tel,
			  i.hr_criminal_mobile,
			  i.hr_criminal_address,
			  i.hr_criminal_pcode,
			  year(i.hr_criminal_birthday),
			  month(i.hr_criminal_birthday),
			  day(i.hr_criminal_birthday),
			  i.hr_criminal_address,
			  i.hr_criminal_sex_type,
			  i.hr_criminal_police_block,
			  i.hr_criminal_police_tel,
			  i.hr_criminal_jobs,
			  i.hr_criminal_house_type_serial_id,
			  i.hr_criminal_police_officer,
			  i.hr_criminal_height_cm,
			  i.hr_criminal_weight_kg,
			  i.hr_criminal_marriage,
			  i.hr_criminal_ankle_cm,
			  i.hr_criminal_house_ping,
			  i.hr_criminal_residence_address,
			  i.hr_criminal_beep_switch,
			  i.hr_criminal_police_mobile,
			  i.serial_id
	   FROM INSERTED as i
	   JOIN hr_prosecution as h on i.hr_prosecution_serial_id = h.serial_id 


WAITFOR DELAY '00:00:03'


INSERT INTO caseprj(hr_criminal_serial_id, 
                    district_prosecutors_serial_id,
                    hr_prosecution_serial_id,
                    datetime_start_year,
                    datetime_start_month,
                    datetime_start_day,
                    datetime_end_year,
                    datetime_end_month,
                    datetime_end_day,
                    tag_monitor_sec,
                    prosecutor_year,
                    prosecutor_type,
                    prosecutor_monitor,
                    datetime_start,
                    datetime_end,
                    electronic_fence,
                    prosecutor_number)

    SELECT h.serial_id,
           h.district_prosecutors_serial_id,
           i.hr_prosecution_serial_id,
           year(i.caseprj_datetime_start),
           month(i.caseprj_datetime_start),
           day(i.caseprj_datetime_start),
           year(i.caseprj_datetime_end),
           month(i.caseprj_datetime_end),
           day(i.caseprj_datetime_end),
           300,
           i.prosecutor_year,
           i.prosecutor_type,
           i.caseprj_monitor_time,
           i.caseprj_datetime_start,
           i.caseprj_datetime_end,
           CONCAT(i.caseprj_no_exit, ';', i.caseprj_no_entry, ';', i.caseprj_entry_notify),
           i.prosecutor_number

    FROM INSERTED as i
    JOIN hr_criminal as h ON i.serial_id = h.application_open_case_serial_id
    
END
GO

### 範例07

In [ ]:
CREATE TRIGGER take_off_insert_for_readtime_trigger ON take_off
FOR INSERT
AS
PRINT '執行 take_off_insert_for_readtime_trigger'

BEGIN

DECLARE @take_off_serial_id int,
        @take_off_district_prosecutors_serial_id int

SELECT　@take_off_serial_id = serial_id FROM INSERTED
SELECT  @take_off_district_prosecutors_serial_id = district_prosecutors_serial_id FROM INSERTED


INSERT INTO take_off_hr_prosecution(take_off_serial_id, hr_prosecution_serial_id)
    SELECT @take_off_serial_id, serial_id
	FROM hr_prosecution
	WHERE 
	    district_prosecutors_serial_id = @take_off_district_prosecutors_serial_id
		and 
		(discharge_flag <> 'Y' and resign_flag <> 'Y' and retire_flag <> 'Y'
		or
		discharge_flag is NULL and resign_flag is NULL and retire_flag is NULL)

END
GO

### 範例08

In [ ]:
CREATE TRIGGER take_off_delete_for_readtime_trigger ON take_off
FOR DELETE
AS
PRINT '執行 take_off_delete_for_readtime_trigger'

DECLARE @take_off_serial_id int
      
SELECT　@take_off_serial_id = serial_id FROM DELETED


DELETE FROM take_off_hr_prosecution
WHERE take_off_serial_id = @take_off_serial_id

GO